In [35]:
LOCAL = False
if not LOCAL:
    from google.colab import drive
    drive.mount('/content/drive')
    from google.colab import userdata
import sys
import os
import json
import pprint
import re
import yaml
import subprocess
from pathlib import Path
from abc import ABC, abstractmethod
from typing import Dict, Any, List, Optional, Union,Tuple # 型ヒントのため
import git # gitpythonライブラリ
import logging # ログ出力用
CURRENT_DIR = "test"
REPO_NAME   = "act"

if LOCAL:
    BASE_PATH = 'H:/マイドライブ/github'
else:
    BASE_PATH = '/content/drive/MyDrive/github'



TEST_REPO_PATH = Path(f'{BASE_PATH}/{REPO_NAME}')
# カレントディレクトリが変更されたか確認
print(f"Current working directory: {os.getcwd()}")

if TEST_REPO_PATH.exists():
    print(f"Path exists: {TEST_REPO_PATH}")
else:
    os.chdir('/content/drive/MyDrive/github')
    !git clone https://github.com/nektos/$REPO_NAME.git


os.chdir(TEST_REPO_PATH)
!git remote -v
!git status
!git shortlog -sn --all

sys.path.append(f'{BASE_PATH}/{CURRENT_DIR}')
os.chdir(f'{BASE_PATH}/{CURRENT_DIR}')


#!git checkout colab-aioserver
#!git branch

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Current working directory: /content/drive/My Drive/github/test
Path exists: /content/drive/MyDrive/github/act
origin	https://github.com/nektos/act.git (fetch)
origin	https://github.com/nektos/act.git (push)
Refresh index: 100% (1890/1890), done.
On branch master
Your branch is up to date with 'origin/master'.

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   .github/actions/choco/entrypoint.sh
	modified:   install.sh
	modified:   pkg/runner/testdata/actions-environment-and-context-tests/docker/entrypoint.sh
	modified:   pkg/runner/testdata/actions/docker-local-noargs/entrypoint.sh
	modified:   pkg/runner/testdata/actions/docker-local/entrypoint.sh
	modified:   pkg/runner/testdata/actions/node12/node_modules/.bin/ncc
	modified:   pkg/

In [30]:


# ロガーの設定 (必要に応じて調整)
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

class BaseRepoScanner(ABC):
    """
    リポジトリの特定情報をスキャン/抽出するための抽象ベースクラス。
    各具象スキャナは、このクラスを継承し、特定の情報を抽出するロジックを実装する。
    """

    def __init__(self, repo_path: str, config: Optional[Dict[str, Any]] = None):
        """
        コンストラクタ。

        Args:
            repo_path (str): 分析対象のローカルリポジトリへのパス。
            config (Optional[Dict[str, Any]]): スキャナ固有の設定（例: 深さ制限、閾値など）。
        """
        self.repo_path = Path(repo_path) # Pathオブジェクトとして扱う
        if not self.repo_path.is_dir() or not (self.repo_path / ".git").is_dir():
            # 実際のプロジェクトでは、より堅牢なGitリポジトリ判定を行う
            # logging.warning(f"Path {repo_path} does not appear to be a valid Git repository root.")
            pass # ここでは警告のみとし、scan()内で詳細なチェックを行うことも可能

        self.config = config if config is not None else {}
        self._results: Optional[Any] = None
        self._status: str = "pending" # "pending", "success", "failure"
        self._error_message: Optional[str] = None
        self.logger = logging.getLogger(self.__class__.__name__)


    @abstractmethod
    def _perform_scan(self) -> Any:
        """
        具象クラスが具体的なスキャン/抽出ロジックを実装するメソッド。
        このメソッドは抽出結果を返す。例外発生時はNoneやエラーを示す値を返すことを想定。
        """
        pass

    def scan(self) -> None:
        """
        スキャン/抽出ロジックを実行し、結果とステータスを設定する。
        """
        self.logger.info(f"Starting scan for {self.repo_path}...")
        try:
            self._results = self._perform_scan()
            if self._results is not None: # _perform_scanがNoneを返さない限り成功とする (空のdict/listも成功)
                self._status = "success"
                self.logger.info("Scan completed successfully.")
            else:
                # _perform_scan が None を返した場合、何らかの理由でデータが取得できなかったとみなす
                # より詳細なエラー処理は _perform_scan 内で行い、例外を発生させるか、
                # あるいはエラーを示す特定の値を返すことを推奨
                self._status = "failure"
                self._error_message = "Scan performed but no results were obtained or an internal error occurred."
                self.logger.warning(self._error_message)

        except Exception as e:
            self._status = "failure"
            self._error_message = f"Exception during scan: {str(e)}"
            self.logger.error(f"Exception in _perform_scan: {e}", exc_info=True)
            self._results = None # エラー時は結果をNoneに

    def get_results(self) -> Any:
        """
        スキャン/抽出結果を返す。
        scan() メソッドが正常に完了した後に呼び出されることを想定。
        """
        if self._status != "success":
            self.logger.warning(f"Scan status is '{self._status}'. Results might be incomplete or absent.")
            if self._error_message:
                self.logger.error(f"Error details: {self._error_message}")
        return self._results

    def _run_git_command(self, command: List[str]) -> Optional[str]:
        """gitコマンドを実行し、標準出力を返すヘルパー関数"""
        try:
            process = subprocess.run(command, cwd=self.repo_path, capture_output=True, text=True, check=True, encoding='utf-8')
            return process.stdout.strip()
        except subprocess.CalledProcessError as e:
            self.logger.error(f"Git command failed: {' '.join(command)}. Error: {e.stderr}")
            return None
        except FileNotFoundError:
            self.logger.error(f"Git command not found. Ensure git is installed and in PATH.")
            return None
    def _get_current_commit_sha(self) -> Optional[str]:
        """現在のコミットSHAを取得するヘルパー関数"""
        return self._run_git_command(["git", "rev-parse", "HEAD"])

    def _get_remote_origin_url(self) -> Optional[str]:
        """リモート 'origin' のURLを取得するヘルパー関数"""
        return self._run_git_command(["git", "config", "--get", "remote.origin.url"])

    def _construct_github_permalink(self, file_path_relative_to_repo: str, commit_sha: Optional[str], raw_content: bool = True) -> Optional[str]:
        """
        GitHubのパーマリンクを構築する。
        raw_content=Trueの場合、raw.githubusercontent.com のURLを生成する。
        """
        if not commit_sha:
            self.logger.warning("Commit SHA not available, cannot construct permalink.")
            return None

        remote_url = self._get_remote_origin_url()
        if remote_url:
            # .git サフィックスを削除
            if remote_url.endswith(".git"):
                remote_url = remote_url[:-4]

            # SSH形式のURLをHTTPS形式に変換
            if remote_url.startswith("git@github.com:"):
                remote_url = remote_url.replace("git@github.com:", "https://github.com/")

            if remote_url.startswith("https://github.com/"):
                # remote_url から owner/repo を抽出
                # 例: https://github.com/owner/repo -> owner/repo
                path_parts = remote_url.split('/')
                if len(path_parts) >= 5:
                    owner = path_parts[3]
                    repo = path_parts[4]
                    if raw_content:
                        return f"https://raw.githubusercontent.com/{owner}/{repo}/{commit_sha}/{str(file_path_relative_to_repo)}"
                    else:
                        return f"https://github.com/{owner}/{repo}/blob/{commit_sha}/{str(file_path_relative_to_repo)}"
        self.logger.warning(f"Could not determine GitHub base URL from remote: {remote_url} for permalink construction.")
        return None

    def get_status(self) -> Dict[str, Optional[str]]:
        """
        スキャンの実行ステータスとエラーメッセージ（あれば）を返す。
        """
        return {
            "status": self._status,
            "error_message": self._error_message
        }

    def report_results(self, indent: int = 2, ensure_ascii: bool = False) -> None:
        """
        スキャン結果を人間が読みやすい形式（デフォルトはJSON）で標準出力に表示する。
        Colab上での即時フィードバック用。
        """
        print(f"--- Scan Report for: {self.__class__.__name__} ---")
        status_info = self.get_status()
        print(f"Target Repository: {self.repo_path.resolve()}")
        print(f"Status: {status_info['status']}")
        if status_info['error_message']:
            print(f"Error Message: {status_info['error_message']}")

        results_data = self.get_results() # scan()が呼ばれていなくても結果は取得できる（Noneや初期値）

        if self._status == "success":
            if results_data is not None:
                print("Results:")
                try:
                    pprint.pp(json.dumps(results_data, ensure_ascii=ensure_ascii, indent=indent, default=str)) # default=strでdatetime等に対応
                except TypeError as e:
                    print(f"Could not serialize results to JSON: {e}")
                    print("Raw results:", results_data)
            else: # 正常終了だが結果がNoneの場合
                 print("Results: No specific data returned (None).")
        elif self._status == "pending":
            print("Scan has not been executed yet.")
        # else: # failure時はエラーメッセージが表示される

        print(f"--- End of Report for: {self.__class__.__name__} ---\n")

    def execute_and_report(self) -> Any:
        """
        scan() を実行し、その結果を report_results() で表示する便利なメソッド。
        最終的な抽出結果を返す。
        """
        self.scan() # エラーハンドリングはscan()内部で行われる
        self.report_results()
        return self.get_results()

# --- ここから具象スキャナクラスの定義例 ---

class RepositoryMetadataScanner(BaseRepoScanner):
    """
    リポジトリ全体の基本的なメタデータ（名前、URL、主要言語、統計情報など）を抽出するスキャナ。
    """
    def __init__(self, repo_path: str, config: Optional[Dict[str, Any]] = None):
        super().__init__(repo_path, config)
        # このスキャナ固有の設定があればここで初期化
        # 例: self.remote_name = self.config.get("remote_name", "origin")


    def _get_remote_url(self) -> Optional[str]:
        """リモートリポジトリのURLを取得する"""
        remote_name = self.config.get("remote_name", "origin")
        return self._run_git_command(["git", "config", "--get", f"remote.{remote_name}.url"])

    def _get_primary_language_guess(self) -> Optional[str]:
        """主要言語を推測する（簡易版：最も多い拡張子など）"""
        # これは非常に簡易的な実装です。より高度なライブラリ(例: linguist)の利用や
        # ファイルサイズを考慮した集計が望ましいです。
        # ここでは、`.git` ディレクトリを除外して拡張子を集計します。
        extension_counts: Dict[str, int] = {}
        for item in self.repo_path.rglob('*'):
            if item.is_file() and not any(part == ".git" for part in item.parts):
                ext = item.suffix.lower()
                if ext:
                    extension_counts[ext] = extension_counts.get(ext, 0) + 1

        if not extension_counts:
            return None

        # 最も多い拡張子を主要言語とする (単純な例)
        primary_ext = max(extension_counts, key=extension_counts.get)
        # 拡張子から言語名へのマッピング (簡易版)
        lang_map = {".py": "Python", ".js": "JavaScript", ".java": "Java", ".ts": "TypeScript", ".go": "Go", ".rb": "Ruby", ".php": "PHP", ".cs": "C#", ".cpp": "C++", ".c": "C", ".md": "Markdown"}
        return lang_map.get(primary_ext, primary_ext) # マップになければ拡張子そのものを返す

    def _get_last_commit_timestamp(self) -> Optional[str]:
        """リポジトリの最新コミット日時 (ISO 8601) を取得する"""
        # %cI は厳密なISO 8601 (例: 2025-05-23T10:30:00+09:00)
        # %aI は author date, %cI は committer date
        return self._run_git_command(["git", "log", "-1", "--format=%cI"])

    def _get_created_at_approx(self) -> Optional[str]:
        """リポジトリの最初のコミット日時 (ISO 8601) を取得する (近似)"""
        # --reverse をつけて最初のコミットを取得
        return self._run_git_command(["git", "log", "--reverse", "--format=%cI", "-n", "1"])


    def _perform_scan(self) -> Optional[Dict[str, Any]]:
        """
        リポジトリメタデータを抽出する具体的なロジック。
        返り値の構造:
        {
            "name": str,
            "url": Optional[str],
            "primary_language": Optional[str],
            "description": Optional[str], // READMEの冒頭などから取得を試みる
            "created_at_approx": Optional[str], // ISO 8601
            "last_push_timestamp": Optional[str], // ISO 8601
            "statistics": {
                "total_file_count": int,
                "total_directory_count": int,
                "total_size_mb_approx": float,
                "total_commits_approx": int,
                "main_contributors_approx": int
            }
        }
        """
        if not self.repo_path.is_dir() or not (self.repo_path / ".git").is_dir():
            self._error_message = f"Path {self.repo_path} is not a valid Git repository."
            self.logger.error(self._error_message)
            return None

        repo_name = self.repo_path.name
        remote_url = self._get_remote_url()
        primary_lang = self._get_primary_language_guess()
        last_commit_ts = self._get_last_commit_timestamp()
        created_at_ts = self._get_created_at_approx()

        # description (READMEから取得を試みる - 簡易版)
        description = None
        readme_patterns = ["README.md", "README.rst", "README.txt", "readme.md"]
        for pattern in readme_patterns:
            readme_file = self.repo_path / pattern
            if readme_file.is_file():
                try:
                    with open(readme_file, 'r', encoding='utf-8', errors='ignore') as f:
                        # 最初の数行または特定のマーカーまでを読むなど、より洗練させることが可能
                        # ここでは最初の250文字を取得
                        content = f.read(250)
                        # 最初の改行までを取得する試み（より短いサマリー）
                        first_newline = content.find('\n')
                        if first_newline != -1:
                            description = content[:first_newline].strip()
                            if description.startswith("#"): # Markdownの見出しの場合、見出しテキストのみ
                                description = description.lstrip("# ").strip()
                        else:
                            description = content.strip()
                        if description: # 何か取得できたらループを抜ける
                            break
                except Exception as e:
                    self.logger.warning(f"Could not read or parse {readme_file}: {e}")
            if description: # ループの外側でもチェック
                break

        # 統計情報
        total_files = 0
        total_dirs = 0
        total_size_bytes = 0
        for item in self.repo_path.rglob('*'):
            if any(part == ".git" for part in item.parts): # .git ディレクトリはスキップ
                continue
            if item.is_file():
                total_files += 1
                try:
                    total_size_bytes += item.stat().st_size
                except OSError:
                    pass # アクセスできないファイルなどはスキップ
            elif item.is_dir():
                total_dirs += 1

        total_size_mb = round(total_size_bytes / (1024 * 1024), 2)

        total_commits_str = self._run_git_command(["git", "rev-list", "--all", "--count"])
        total_commits = int(total_commits_str) if total_commits_str and total_commits_str.isdigit() else 0

        # 主要コントリビュータ数 (簡易版: 上位5名など)
        # `git shortlog -sn --all` の出力をパース
        shortlog_output = self._run_git_command(["git", "shortlog", "-sn", "--all"])
        main_contributors = 0
        if shortlog_output:
            main_contributors = len(shortlog_output.splitlines()) # 行数でカウント (非常に単純)
            # より正確には、上位N名などをカウントする

        return {
            "name": repo_name,
            "url": remote_url,
            "primary_language": primary_lang,
            "description": description,
            "created_at_approx": created_at_ts,
            "last_push_timestamp": last_commit_ts,
            "statistics": {
                "total_file_count": total_files,
                "total_directory_count": total_dirs,
                "total_size_mb_approx": total_size_mb,
                "total_commits_approx": total_commits,
                "main_contributors_approx": main_contributors # ここはより洗練させる余地あり
            }
        }

# --- 以降、他の具象スキャナクラス（DirectoryTreeScannerなど）のスタブを同様に定義していく ---
# class DirectoryTreeScanner(BaseRepoScanner):
#     def _perform_scan(self) -> Optional[Dict[str, Any]]:
#         # ディレクトリツリー構造を抽出するロジック
#         # self.config['depth_limit'], self.config['file_count_threshold'] などを利用
#         pass

# class DocumentationFileScanner(BaseRepoScanner):
#     def _perform_scan(self) -> Optional[List[Dict[str, Any]]]:
#         # README等のドキュメントファイル情報を抽出するロジック
#         # self.config['doc_file_patterns'], self.config['snippet_length'] などを利用
#         pass

# class ConfigurationFileScanner(BaseRepoScanner):
#     def _perform_scan(self) -> Optional[List[Dict[str, Any]]]:
#         # 構成ファイル情報を抽出するロジック
#         # self.config['target_config_descriptors'] などを利用
#         pass


# --- Colabでの実行例 ---
if __name__ == '__main__':
    # このスクリプトをColabに貼り付けて実行する場合、
    # repo_path はColab環境内の実際のGitリポジトリのパスを指定する必要があります。
    # 例: Google Driveをマウントしている場合
    # REPO_ROOT_PATH = "/content/drive/MyDrive/path/to/your/cloned_repo"

    # ローカルでテストする場合 (このスクリプトと同じ階層に .git があると仮定)
    # あるいは、テストしたいリポジトリのパスを絶対パスで指定
    CURRENT_SCRIPT_DIR = Path(os.getcwd())
    # TEST_REPO_PATH = CURRENT_SCRIPT_DIR # カレントディレクトリがGitリポジトリのルートであると仮定
    # より具体的にテストリポジトリを指定する方が良い
    # 例: TEST_REPO_PATH = Path('/content/drive/MyDrive/github/act')

    # ユーザーにテストリポジトリのパスを入力させるか、固定のパスを使用
    #test_repo_input_path = input(f"Enter the path to a local Git repository for testing (e.g., {CURRENT_SCRIPT_DIR}): ")

    if not TEST_REPO_PATH.is_dir() or not (TEST_REPO_PATH / ".git").is_dir():
        print(f"Error: The path '{TEST_REPO_PATH}' is not a valid Git repository. Please provide a valid path.")
    else:
        print(f"\n--- Testing RepositoryMetadataScanner for: {TEST_REPO_PATH} ---")
        metadata_config = {"remote_name": "origin"} # 例: リモート名指定
        metadata_scanner = RepositoryMetadataScanner(str(TEST_REPO_PATH), config=metadata_config)
        metadata_results = metadata_scanner.execute_and_report()

        # 期待される出力の型（例）
        # if metadata_results:
        #     assert isinstance(metadata_results.get("name"), str)
        #     assert isinstance(metadata_results.get("statistics", {}).get("total_file_count"), int)
        #     print("\n(Sample assertion checks would go here for Pytest)")

        # print(f"\n--- Example of how to use other scanners (stubs for now) ---")
        # tree_scanner_config = {"depth_limit": 3, "file_count_threshold": 50}
        # tree_scanner = DirectoryTreeScanner(str(TEST_REPO_PATH), config=tree_scanner_config)
        # tree_results = tree_scanner.execute_and_report() # 現状は _perform_scan が未実装なのでNoneが返る

        # doc_scanner_config = {"snippet_length": 300}
        # doc_scanner = DocumentationFileScanner(str(TEST_REPO_PATH), config=doc_scanner_config)
        # doc_results = doc_scanner.execute_and_report()

        # config_scanner_config = {
        #     "target_config_descriptors": [
        #         {"filename_pattern": "package.json", "file_type_label": "NPM_PackageConfig", "key_info_extractors": ["dependencies", "scripts"]},
        #         {"filename_pattern": "Dockerfile", "file_type_label": "Docker_Config", "key_info_extractors": ["FROM", "EXPOSE"]}
        #     ]
        # }
        # config_scanner = ConfigurationFileScanner(str(TEST_REPO_PATH), config=config_scanner_config)
        # config_results = config_scanner.execute_and_report()

        # --- 最終的にこれらの結果を統合して initialRepositoryScanData を構築する ---
        # initial_scan_data = {
        #     "repositoryInfo": metadata_results,
        #     "directoryStructure": tree_results, # DirectoryTreeScannerの結果
        #     "documentationFiles": doc_results, # DocumentationFileScannerの結果
        #     "configurationFiles": config_results, # ConfigurationFileScannerの結果
        #     # ... 他のスキャン結果もここに追加 ...
        # }
        # print("\n--- Example of combined initial_scan_data (conceptual) ---")
        # print(json.dumps(initial_scan_data, indent=2, default=str, ensure_ascii=False))


--- Testing RepositoryMetadataScanner for: /content/drive/MyDrive/github/act ---
--- Scan Report for: RepositoryMetadataScanner ---
Target Repository: /content/drive/MyDrive/github/act
Status: success
Results:
('{\n'
 '  "name": "act",\n'
 '  "url": "https://github.com/nektos/act.git",\n'
 '  "primary_language": "JavaScript",\n'
 '  "description": '
 '"![act-logo](https://raw.githubusercontent.com/wiki/nektos/act/img/logo-150.png)",\n'
 '  "created_at_approx": "2025-05-22T21:57:52+00:00",\n'
 '  "last_push_timestamp": "2025-05-22T21:57:52+00:00",\n'
 '  "statistics": {\n'
 '    "total_file_count": 1890,\n'
 '    "total_directory_count": 529,\n'
 '    "total_size_mb_approx": 43.06,\n'
 '    "total_commits_approx": 1396,\n'
 '    "main_contributors_approx": 213\n'
 '  }\n'
 '}')
--- End of Report for: RepositoryMetadataScanner ---



In [ ]:
class DirectoryTreeScanner(BaseRepoScanner):
    """
    リポジトリのディレクトリツリー構造を抽出するスキャナ。
    設定で深さ制限やファイル数閾値を指定可能。
    """
    def __init__(self, repo_path: str, config: Optional[Dict[str, Any]] = None):
        super().__init__(repo_path, config)
        self.depth_limit = self.config.get("depth_limit", None) # Noneの場合は制限なし
        self.file_count_threshold = self.config.get("file_count_threshold", None) # Noneの場合は制限なし

    def _scan_directory_recursive(self, current_path: Path, current_depth: int) -> Dict[str, Any]:
        """
        再帰的にディレクトリをスキャンし、ツリー構造のノードを構築する。
        """
        node_name = current_path.name
        relative_path_str = str(current_path.relative_to(self.repo_path))

        node_info: Dict[str, Any] = {
            "name": node_name,
            "type": "directory",
            "path": relative_path_str if relative_path_str != "." else ".", # ルートは"."
            "children": []
        }

        # ファイル数閾値のチェック
        if self.file_count_threshold is not None:
            try:
                num_files_in_dir = sum(1 for item in current_path.iterdir() if item.is_file() and not any(part == ".git" for part in item.parts))
                if num_files_in_dir > self.file_count_threshold:
                    node_info["status"] = "exceeds_threshold"
                    node_info["file_count"] = num_files_in_dir
                    # 閾値を超えたら、このディレクトリの子供の展開は行わない (または限定的にする)
                    # ここでは子供の展開をスキップする
                    return node_info
            except OSError as e:
                self.logger.warning(f"Could not count files in {current_path}: {e}")
                # アクセスできないディレクトリはエラーとして扱うか、空のchildrenとする
                node_info["status"] = "access_error"
                return node_info


        # 深さ制限のチェック
        if self.depth_limit is not None and current_depth >= self.depth_limit:
            # 深さ制限に達したら、これ以上子供を展開しない
            # (オプション) 制限に達したことを示すフラグを立てることも可能
            # node_info["status"] = "depth_limit_reached"
            return node_info

        try:
            for item in sorted(current_path.iterdir()): # 名前順でソート
                if any(part == ".git" for part in item.parts): # .git ディレクトリはスキップ
                    continue

                item_relative_path_str = str(item.relative_to(self.repo_path))
                if item.is_dir():
                    child_node = self._scan_directory_recursive(item, current_depth + 1)
                    node_info["children"].append(child_node)
                elif item.is_file():
                    try:
                        file_size = item.stat().st_size
                    except OSError:
                        file_size = -1 # アクセスできない場合は-1など

                    node_info["children"].append({
                        "name": item.name,
                        "type": "file",
                        "path": item_relative_path_str,
                        "size_bytes": file_size
                    })
        except OSError as e:
            self.logger.warning(f"Could not iterate directory {current_path}: {e}")
            node_info["status"] = "iteration_error"
            # この場合もchildrenは空のまま

        return node_info

    def _perform_scan(self) -> Optional[Dict[str, Any]]:
        """
        ディレクトリツリー構造を抽出する具体的なロジック。
        返り値の構造 (ネストされたDict):
        {
          "name": ".", "type": "directory", "path": ".", "children": [
            {"name": "src", "type": "directory", "path": "src", "children": [...]},
            {"name": "docs", "type": "directory", "path": "docs", "status": "exceeds_threshold", "file_count": 120},
            {"name": "README.md", "type": "file", "path": "README.md", "size_bytes": 1024}
          ]
        }
        """
        if not self.repo_path.is_dir(): # .gitのチェックはBaseで行うか、ここでも行うか
            self._error_message = f"Path {self.repo_path} is not a valid directory."
            self.logger.error(self._error_message)
            return None

        try:
            # ルートディレクトリからスキャン開始
            root_node = self._scan_directory_recursive(self.repo_path, current_depth=0)
            return root_node
        except Exception as e:
            self._error_message = f"Error during directory tree scan: {str(e)}"
            self.logger.error(self._error_message, exc_info=True)
            return None



# --- Colabでの実行例 ---
if __name__ == '__main__':

    if not TEST_REPO_PATH.is_dir() or not (TEST_REPO_PATH / ".git").is_dir():
        print(f"Error: The path '{TEST_REPO_PATH}' is not a valid Git repository. Please provide a valid path.")
    else:
        print(f"\n--- Testing RepositoryMetadataScanner for: {TEST_REPO_PATH} ---")

        print(f"\n--- Testing DirectoryTreeScanner for: {TEST_REPO_PATH} ---")
        # depth_limit: Noneで無制限、file_count_threshold: Noneで無制限
        tree_scanner_config = {"depth_limit": 2, "file_count_threshold": 100}
        tree_scanner = DirectoryTreeScanner(str(TEST_REPO_PATH), config=tree_scanner_config)
        tree_results = tree_scanner.execute_and_report()

        # print(f"\n--- Example of how to use other scanners (stubs for now) ---")
        # doc_scanner_config = {"snippet_length": 300}
        # doc_scanner = DocumentationFileScanner(str(TEST_REPO_PATH), config=doc_scanner_config)
        # doc_results = doc_scanner.execute_and_report()

        # config_scanner_config = {
        #     "target_config_descriptors": [
        #         {"filename_pattern": "package.json", "file_type_label": "NPM_PackageConfig", "key_info_extractors": ["dependencies", "scripts"]},
        #         {"filename_pattern": "Dockerfile", "file_type_label": "Docker_Config", "key_info_extractors": ["FROM", "EXPOSE"]}
        #     ]
        # }
        # config_scanner = ConfigurationFileScanner(str(TEST_REPO_PATH), config=config_scanner_config)
        # config_results = config_scanner.execute_and_report()

        # --- 最終的にこれらの結果を統合して initialRepositoryScanData を構築する ---
        # initial_scan_data = {
        #     "repositoryInfo": metadata_results,
        #     "directoryStructure": tree_results,
        #     "documentationFiles": doc_results,
        #     "configurationFiles": config_results,
        # }
        # print("\n--- Example of combined initial_scan_data (conceptual) ---")
        # if metadata_results and tree_results: # 少なくとも主要な2つが成功した場合
        #     initial_scan_data = {
        #         "repositoryInfo": metadata_results,
        #         "directoryStructure": tree_results
        #         # 他のスキャナの結果もここに追加
        #     }
        #     print(json.dumps(initial_scan_data, indent=2, default=str, ensure_ascii=False))


In [ ]:
class DocumentationFileScanner(BaseRepoScanner):
    """
    リポジトリ内の主要なドキュメントファイル（README*, CONTRIBUTING*, LICENSEなど）を検索し、
    各ファイルのパス、パーマリンク、サイズ、内容の冒頭抜粋を抽出するスキャナ。
    """
    def __init__(self, repo_path: str, config: Optional[Dict[str, Any]] = None):
        super().__init__(repo_path, config)
        # 設定可能なデフォルト値
        self.doc_file_patterns = self.config.get(
            "doc_file_patterns",
            [
                r"README(?:\.md|\.rst|\.txt)?$", # README.md, README.rst, README.txt, README
                r"CONTRIBUTING(?:\.md|\.rst|\.txt)?$",
                r"CODE_OF_CONDUCT(?:\.md|\.rst|\.txt)?$",
                r"CHANGELOG(?:\.md|\.rst|\.txt)?$",
                r"LICENSE(?:\.md|\.rst|\.txt|)$", # LICENSE, LICENSE.md など
                r"SECURITY\.md$"
            ]
        )
        self.snippet_length = self.config.get("snippet_length", 500) # 文字数
        self.current_commit_sha = None # _perform_scanの最初で取得

    def _find_matching_files(self) -> List[Path]:
        """
        設定されたパターンに一致するファイルをリポジトリ内から検索する。
        大文字・小文字を区別しない検索を行う。
        """
        matching_files: List[Path] = []
        compiled_patterns = [re.compile(pattern, re.IGNORECASE) for pattern in self.doc_file_patterns]

        for item in self.repo_path.rglob('*'):
            if any(part == ".git" for part in item.parts): # .git ディレクトリはスキップ
                continue
            if item.is_file():
                for pattern_re in compiled_patterns:
                    if pattern_re.search(item.name):
                        matching_files.append(item)
                        break # 一致したら次のファイルへ
        return matching_files

    def _extract_content_snippet(self, file_path: Path) -> Optional[str]:
        """
        ファイルから指定文字数の内容抜粋を取得する。
        """
        try:
            with open(file_path, 'r', encoding='utf-8', errors='ignore') as f:
                snippet = f.read(self.snippet_length)
                # TODO: READMEの場合、最初の数セクションの見出しと要約、といったより高度な抜粋も検討
                return snippet.strip()
        except Exception as e:
            self.logger.warning(f"Could not read or extract snippet from {file_path}: {e}")
            return None

    def _get_license_type_from_content(self, content: Optional[str]) -> Optional[str]:
        """
        (非常に簡易的な)ライセンス内容からのSPDX識別子推測。
        実際にはより堅牢なライブラリ (例: scancode-toolkitの一部など) の利用を検討。
        """
        if not content:
            return None
        content_lower = content.lower()
        if "mit license" in content_lower:
            return "MIT"
        if "apache license" in content_lower and "version 2.0" in content_lower:
            return "Apache-2.0"
        if "gnu general public license" in content_lower:
            if "version 3" in content_lower:
                return "GPL-3.0-only" # or or-later
            if "version 2" in content_lower:
                return "GPL-2.0-only" # or or-later
            return "GPL" # Version不明
        if "bsd 3-clause license" in content_lower:
            return "BSD-3-Clause"
        # 他の主要ライセンスも追加可能
        return None


    def _perform_scan(self) -> Optional[List[Dict[str, Any]]]:
        """
        ドキュメントファイル情報を抽出する具体的なロジック。
        返り値の構造 (List[Dict[str, Any]]):
        [
            {
                "path": str, # ルートからの相対パス
                "filename": str,
                "permalink": Optional[str], # GitHub等のWeb UIへのパーマリンク (raw content URL)
                "size_bytes": int,
                "content_snippet": Optional[str], # ファイル冒頭の抜粋
                "detected_license_type": Optional[str] # LICENSEファイルの場合のみ
            }, ...
        ]
        """
        if not self.repo_path.is_dir():
            self._error_message = f"Path {self.repo_path} is not a valid directory."
            self.logger.error(self._error_message)
            return None

        self.current_commit_sha = self._get_current_commit_sha()
        if not self.current_commit_sha:
            self.logger.warning("Could not determine current commit SHA. Permalinks will not be generated.")

        found_documents: List[Dict[str, Any]] = []
        discovered_files = self._find_matching_files()

        for file_path_obj in discovered_files:
            relative_path_str = str(file_path_obj.relative_to(self.repo_path))
            # raw_content=True を指定して raw URL を取得
            permalink = self._construct_github_permalink(relative_path_str, self.current_commit_sha, raw_content=True)

            try:
                size_bytes = file_path_obj.stat().st_size
            except OSError as e:
                self.logger.warning(f"Could not get size for {file_path_obj}: {e}")
                size_bytes = -1 # エラーを示す値

            content_snippet = self._extract_content_snippet(file_path_obj)

            doc_info: Dict[str, Any] = {
                "path": relative_path_str,
                "filename": file_path_obj.name,
                "permalink": permalink,
                "size_bytes": size_bytes,
                "content_snippet": content_snippet,
            }

            # LICENSEファイルの場合、簡易的なライセンスタイプ検出を試みる
            if "LICENSE" in file_path_obj.name.upper(): # LICENSE, License.md などにマッチ
                doc_info["detected_license_type"] = self._get_license_type_from_content(content_snippet)

            found_documents.append(doc_info)

        return found_documents

# class ConfigurationFileScanner(BaseRepoScanner):
#     def _perform_scan(self) -> Optional[List[Dict[str, Any]]]:
#         # 構成ファイル情報を抽出するロジック
#         # self.config['target_config_descriptors'] などを利用
#         pass


# --- Colabでの実行例 ---
if __name__ == '__main__':

    if not TEST_REPO_PATH.is_dir() or not (TEST_REPO_PATH / ".git").is_dir():
        print(f"Error: The path '{TEST_REPO_PATH}' is not a valid Git repository. Please provide a valid path.")
    else:
        # --- Testing DocumentationFileScanner ---
        print(f"\n--- Testing DocumentationFileScanner for: {TEST_REPO_PATH} ---")
        doc_scanner_config = {
            "snippet_length": 200,
        }
        doc_scanner = DocumentationFileScanner(str(TEST_REPO_PATH), config=doc_scanner_config)
        doc_results = doc_scanner.execute_and_report()

        if doc_results:
            print(f"\nFound {len(doc_results)} documentation file(s).")


In [36]:

class ConfigurationFileScanner(BaseRepoScanner):
    """
    リポジトリ内の主要な構成ファイルを検索し、各ファイルのパス、パーマリンク、
    ファイルタイプ、そしてファイル内容から抽出されたキー情報を取得するスキャナ。
    """
    def __init__(self, repo_path: str, config: Optional[Dict[str, Any]] = None):
        super().__init__(repo_path, config)
        # 設定可能なデフォルト値
        # target_config_descriptors: 検索・解析対象の構成ファイル記述子のリスト。
        # 各記述子は以下のような情報を持つ:
        # {"filename_pattern": "package.json", "file_type_label": "NPM_PackageConfig",
        #  "key_info_extractors": {"dependencies": "object", "scripts": "object", "version": "string"}, "parser_type": "json"}
        # {"filename_pattern": "Dockerfile", "file_type_label": "Docker_Config",
        #  "key_info_extractors": {"FROM": "line", "EXPOSE": "line_multi", "CMD": "line"}, "parser_type": "dockerfile"}
        # {"filename_pattern": r"\.yml$|\.yaml$", "directory_pattern": r"\.github[/\\]workflows", "file_type_label": "GitHub_Workflow",
        #  "key_info_extractors": {"name": "string", "on": "object", "jobs": "object_keys"}, "parser_type": "yaml"}
        self.target_config_descriptors = self.config.get("target_config_descriptors", [])
        self.current_commit_sha = None # _perform_scanの最初で取得

    def _find_matching_files(self) -> List[Tuple[Path, Dict[str, Any]]]:
        """
        設定された記述子に一致するファイルをリポジトリ内から検索する。
        各ファイルに対応する記述子も一緒に返す。
        """
        matching_files_with_descriptors: List[Tuple[Path, Dict[str, Any]]] = []

        for descriptor in self.target_config_descriptors:
            filename_pattern_str = descriptor.get("filename_pattern")
            directory_pattern_str = descriptor.get("directory_pattern") # オプション

            if not filename_pattern_str:
                self.logger.warning(f"Skipping descriptor due to missing 'filename_pattern': {descriptor}")
                continue

            try:
                filename_re = re.compile(filename_pattern_str, re.IGNORECASE)
                dir_re = re.compile(directory_pattern_str, re.IGNORECASE) if directory_pattern_str else None
            except re.error as e:
                self.logger.error(f"Invalid regex in descriptor '{descriptor.get('file_type_label', 'Unknown')}': {e}")
                continue

            for item in self.repo_path.rglob('*'): # rglobは再帰的に検索
                if any(part == ".git" for part in item.parts):
                    continue
                if item.is_file():
                    if filename_re.search(item.name):
                        if dir_re: # ディレクトリパターンも指定されている場合
                            # Pathオブジェクトを文字列に変換してから正規表現検索
                            if dir_re.search(str(item.parent.relative_to(self.repo_path))):
                                matching_files_with_descriptors.append((item, descriptor))
                        else: # ファイル名パターンのみ
                            matching_files_with_descriptors.append((item, descriptor))
        return matching_files_with_descriptors

    def _parse_file_content(self, file_path: Path, parser_type: str) -> Any:
        """
        ファイル内容を指定されたパーサータイプでパースする。
        """
        try:
            with open(file_path, 'r', encoding='utf-8', errors='ignore') as f:
                content = f.read()
                if parser_type == "json":
                    return json.loads(content)
                elif parser_type == "yaml":
                    return yaml.safe_load(content)
                elif parser_type == "dockerfile": # Dockerfileは行ベースで特定の指示を抽出
                    lines = content.splitlines()
                    docker_info = {}
                    for line in lines:
                        line_strip = line.strip()
                        if line_strip.startswith("FROM"):
                            docker_info.setdefault("FROM", []).append(line_strip.split(maxsplit=1)[1] if len(line_strip.split(maxsplit=1)) > 1 else "")
                        elif line_strip.startswith("EXPOSE"):
                             docker_info.setdefault("EXPOSE", []).append(line_strip.split(maxsplit=1)[1] if len(line_strip.split(maxsplit=1)) > 1 else "")
                        elif line_strip.startswith("CMD"):
                             docker_info.setdefault("CMD", []).append(line_strip.split(maxsplit=1)[1] if len(line_strip.split(maxsplit=1)) > 1 else "")
                        # 他のDocker命令も必要に応じて追加
                    return docker_info
                # 他のパーサータイプも追加可能 (例: ini, xml, properties)
                else: # 不明なパーサータイプの場合は生の内容を返すか、エラーとする
                    self.logger.warning(f"Unknown parser_type '{parser_type}' for {file_path}. Returning raw content for key info extraction.")
                    return content # または None
        except json.JSONDecodeError as e:
            self.logger.error(f"JSON parsing error for {file_path}: {e}")
            return None
        except yaml.YAMLError as e:
            self.logger.error(f"YAML parsing error for {file_path}: {e}")
            return None
        except Exception as e:
            self.logger.error(f"Error reading or parsing file {file_path}: {e}")
            return None

    def _extract_key_info(self, parsed_content: Any, extractors: Dict[str, str], parser_type: str) -> Dict[str, Any]:
        """
        パースされた内容から、指定されたキー情報を抽出する。
        extractors: {"key_to_extract": "expected_type_or_method", ...}
        expected_type_or_method: "string", "object", "list", "object_keys", "line", "line_multi"
        """
        key_info: Dict[str, Any] = {}
        if parsed_content is None:
            return key_info

        for key, method in extractors.items():
            try:
                if parser_type == "dockerfile" and isinstance(parsed_content, dict): # Dockerfileパーサーの結果
                    if method == "line": # 最初の行
                        key_info[key] = parsed_content.get(key, [])[0] if parsed_content.get(key) else None
                    elif method == "line_multi": # 全ての行 (リスト)
                        key_info[key] = parsed_content.get(key)
                    else:
                        key_info[key] = parsed_content.get(key)

                elif isinstance(parsed_content, dict): # JSON, YAML (dict)
                    if method == "object_keys": # jobs: object_keys -> job名のリスト
                        value = parsed_content.get(key)
                        key_info[key] = list(value.keys()) if isinstance(value, dict) else None
                    else: # string, object, list
                        key_info[key] = parsed_content.get(key)

                elif isinstance(parsed_content, list) and parser_type == "yaml": # YAMLでルートがリストの場合
                     # このケースは現状のextractorsでは扱いにくい。より複雑なパス指定が必要。
                     # 例えば、特定要素のキーを抽出するなど。今回は単純化のためスキップ。
                    self.logger.debug(f"Key info extraction for root list YAML not fully supported for key '{key}'.")
                    pass

                # ここで型チェックや変換を行うことも可能
                # if method == "string" and not isinstance(key_info.get(key), str):
                #     key_info[key] = str(key_info.get(key)) if key_info.get(key) is not None else None

            except Exception as e:
                self.logger.warning(f"Could not extract key '{key}' using method '{method}' from content: {e}")
                key_info[key] = None
        return key_info

    def _perform_scan(self) -> Optional[List[Dict[str, Any]]]:
        """
        構成ファイル情報を抽出する具体的なロジック。
        返り値の構造 (List[Dict[str, Any]]):
        [
            {
                "path": str, # ルートからの相対パス
                "filename": str,
                "fileType": str, # 記述子で指定されたラベル
                "permalink": Optional[str], # GitHub等のWeb UIへのパーマリンク (raw content URL)
                "keyInfoExtracted": Dict[str, Any] # 抽出されたキー情報
            }, ...
        ]
        """
        if not self.repo_path.is_dir():
            self._error_message = f"Path {self.repo_path} is not a valid directory."
            self.logger.error(self._error_message)
            return None

        self.current_commit_sha = self._get_current_commit_sha()
        if not self.current_commit_sha:
            self.logger.warning("Could not determine current commit SHA. Permalinks will not be generated.")

        found_configs: List[Dict[str, Any]] = []
        discovered_files_with_descriptors = self._find_matching_files()

        for file_path_obj, descriptor in discovered_files_with_descriptors:
            relative_path_str = str(file_path_obj.relative_to(self.repo_path))
            permalink = self._construct_github_permalink(relative_path_str, self.current_commit_sha, raw_content=True)

            file_type_label = descriptor.get("file_type_label", "UnknownConfig")
            parser_type = descriptor.get("parser_type", "text") # デフォルトはテキストとして扱う
            key_info_extractors = descriptor.get("key_info_extractors", {})

            parsed_content = self._parse_file_content(file_path_obj, parser_type)
            key_info = self._extract_key_info(parsed_content, key_info_extractors, parser_type)

            config_info: Dict[str, Any] = {
                "path": relative_path_str,
                "filename": file_path_obj.name,
                "fileType": file_type_label,
                "permalink": permalink,
                "keyInfoExtracted": key_info,
            }
            found_configs.append(config_info)

        return found_configs


# --- Colabでの実行例 ---
if __name__ == '__main__':

    if not TEST_REPO_PATH.is_dir() or not (TEST_REPO_PATH / ".git").is_dir():
        print(f"Error: The path '{TEST_REPO_PATH}' is not a valid Git repository. Please provide a valid path.")
    else:
        # --- Testing ConfigurationFileScanner ---
        print(f"\n--- Testing ConfigurationFileScanner for: {TEST_REPO_PATH} ---")
        config_scanner_config = {
            "target_config_descriptors": [
                {
                    "filename_pattern": r"package\.json$",
                    "file_type_label": "NPM_PackageConfig",
                    "key_info_extractors": {
                        "name": "string",
                        "version": "string",
                        "description": "string",
                        "dependencies": "object",
                        "devDependencies": "object",
                        "scripts": "object_keys" # scriptsオブジェクトのキー（スクリプト名）をリストで取得
                    },
                    "parser_type": "json"
                },
                {
                    "filename_pattern": r"Dockerfile$",
                    "file_type_label": "Docker_Config",
                    "key_info_extractors": {
                        "FROM": "line", # 最初のFROM命令
                        "EXPOSE": "line_multi", # 全てのEXPOSE命令（リスト）
                        "CMD": "line"  # 最後のCMD命令 (もし複数あれば最後のものが有効になることが多い)
                    },
                    "parser_type": "dockerfile"
                },
                {
                    "filename_pattern": r"\.yml$|\.yaml$",
                    "directory_pattern": r"\.github[/\\]workflows", # .github/workflows 内のYAMLファイル
                    "file_type_label": "GitHub_Workflow",
                    "key_info_extractors": {
                        "name": "string", # workflow名
                        "on": "object", # トリガー条件
                        "jobs": "object_keys" # job名のリスト
                    },
                    "parser_type": "yaml"
                },
                { # actリポジトリ用の設定
                    "filename_pattern": r"\.golangci\.yml$",
                    "file_type_label": "GolangCI_Config",
                    "key_info_extractors": {"run": "object", "linters-settings": "object", "issues": "object"},
                    "parser_type": "yaml"
                },
                {
                    "filename_pattern": r"\.goreleaser\.yml$",
                    "file_type_label": "GoReleaser_Config",
                    "key_info_extractors": {"builds": "list", "archives": "list", "checksum": "object"},
                    "parser_type": "yaml"
                }
            ]
        }
        config_scanner = ConfigurationFileScanner(str(TEST_REPO_PATH), config=config_scanner_config)
        config_results = config_scanner.execute_and_report()

        if config_results:
            print(f"\nFound {len(config_results)} configuration file(s) matching descriptors.")



--- Testing ConfigurationFileScanner for: /content/drive/MyDrive/github/act ---
--- Scan Report for: ConfigurationFileScanner ---
Target Repository: /content/drive/MyDrive/github/act
Status: success
Results:
('[\n'
 '  {\n'
 '    "path": "pkg/runner/testdata/actions/node12/package.json",\n'
 '    "filename": "package.json",\n'
 '    "fileType": "NPM_PackageConfig",\n'
 '    "permalink": '
 '"https://raw.githubusercontent.com/nektos/act/bd4bc99ec4ddf633e73e69baf0d7f7b9bbef880b/pkg/runner/testdata/actions/node12/package.json",\n'
 '    "keyInfoExtracted": {\n'
 '      "name": "node12",\n'
 '      "version": "1.0.0",\n'
 '      "description": "",\n'
 '      "dependencies": {\n'
 '        "@actions/core": "^1.2.6",\n'
 '        "@actions/github": "^4.0.0"\n'
 '      },\n'
 '      "devDependencies": {\n'
 '        "@vercel/ncc": "^0.24.1"\n'
 '      },\n'
 '      "scripts": [\n'
 '        "test",\n'
 '        "build"\n'
 '      ]\n'
 '    }\n'
 '  },\n'
 '  {\n'
 '    "path": '
 '"pkg/runn